<a href="https://colab.research.google.com/github/BIwithRuben/vulnerabilidad_climatica_mex/blob/Marcio/TEC_UCM_Notebook_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marcio's notebook for the project ENIGH TEC-UCM

Files are too large, some of them with more than 500Mb.  
I used python's library POLARS, and confirmed if it is more efficient than pandas in terms of memory usage and performnace.

#### CONSIDERANDO ARCHIVOS EN MI ORDENADORA (sin API)

#### IMPORTING LIBRARIES

In [75]:
import pandas as pd
import polars as pl

#### CODE con libreria POLARS

In [76]:
# Reading files with POLARS

file_name = 'hogares.csv'  # concentradohogar, erogaciones, gastoshogar, hogares, ingresos, viviendas

df = pl.read_csv(
    file_name,
    infer_schema_length=10000,
    schema_overrides={"tipo_viv": pl.Utf8},
    #ignore_errors=True,
    null_values="&"
)

print(df.shape)
print(df.head())
print(df.schema)
print(f'Columns {df.columns}')

(91414, 148)
shape: (5, 148)
┌───────────┬──────────┬───────────┬────────────┬───┬─────────┬─────────┬─────┬────────┐
│ folioviv  ┆ foliohog ┆ huespedes ┆ huesp_come ┆ … ┆ entidad ┆ est_dis ┆ upm ┆ factor │
│ ---       ┆ ---      ┆ ---       ┆ ---        ┆   ┆ ---     ┆ ---     ┆ --- ┆ ---    │
│ i64       ┆ i64      ┆ i64       ┆ str        ┆   ┆ i64     ┆ i64     ┆ i64 ┆ i64    │
╞═══════════╪══════════╪═══════════╪════════════╪═══╪═════════╪═════════╪═════╪════════╡
│ 100001901 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100001902 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100001904 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100001905 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100002501 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 2       ┆ 2   ┆ 196    │
└───────────┴──────────┴───────────┴────────────┴───┴─────────┴─────────┴─────┴──

#### SELECCIÓN DE VARIABLES



In [77]:
# Select specific columns

# FROM viviendas.csv
#selected = df.select(['folioviv', 'tot_resid', 'mat_pared', 'mat_techos', 'ab_agua', 'agua_noe', 'dotac_agua', 'sanit_agua', 'drenaje'])

# FROM concentradohogar.csv
#selected = df.select(['folioviv', 'foliohog', 'ing_cor', 'gasto_mon', 'ubica_geo', 'tam_loc'])

# FROM erogaciones.csv
#selected = df.select(['folioviv', 'foliohog', 'ing_cor', 'gasto_mon', 'ubica_geo', 'tam_loc'])

# FROM hogares.csv
selected = df.select(['folioviv',
                      'acc_alim2',
                      'acc_alim7',
                      'acc_alim9',
                      'celular',
                      'tsalud1_h',
                      'tsalud1_m',
                      'camb_clim',
                      'f_sequia',
                      'f_inunda',
                      'f_helada',
                      'f_incendio',
                      'f_huracan',
                      'f_desliza',
                      'f_otro',
                      'af_viv',
                      'af_empleo',
                      'af_cultivo',
                      'af_trabajo',
                      'af_salud',
                      'af_otro'
                    ]
                  )


### Variables by Relevant Information

#### CLAVE PRIMARIA (Primary Key)
| Variable | Definition |
| :--- | :--- |
| **folioviv** | Identificador de la vivienda compuesto por dos dígitos con la clave de la entidad federativa, uno con el ámbito (urbano, código diferente a 6; rural, código 6), cuatro dígitos del número consecutivo de la UPM, un dígito de la decena de levantamiento y dos dígitos con un número consecutivo para la vivienda seleccionada. |

---

##### Seguridad alimentaria y nutricional (LEVÉ, MODERADA Y GRAVE) (Food and Nutritional Security (MILD, MODERATE, AND SEVERE))

- **Sin comida (acc_alim2)**
  - Type: **BOOLEAN**
  - Definition: En los últimos tres meses, por falta de dinero o recursos ¿alguna vez se quedaron sin comida? (In the last three months, due to lack of money or resources, did you ever run out of food?)
- **Sintió hambre y no comió (acc_alim7)**
  - Type: **BOOLEAN**
  - Definition: En los últimos tres meses, por falta de dinero o recursos, ¿alguna vez usted o alguna persona adulta en su hogar sintió hambre pero no comió? (In the last three months, due to lack of money or resources, did you or any adult in your household ever feel hungry but not eat?)
- **Mendigar por comida (acc_alim9)**
  - Type: **BOOLEAN**
  - Definition: En los últimos tres meses, por falta de dinero o recursos ¿alguna vez tuvieron que hacer algo que hubieran preferido no hacer para conseguir comida, tal como mendigar (pedir limosna) o mandar a los niños a trabajar? (In the last three months, due to lack of money or resources, did you ever have to do something you would have preferred not to do to get food, such as begging or sending children to work?)

---

#### Vulnerabilidad económica / Inclusión digital (Economic Vulnerability / Digital Inclusion)

- **Teléfono celular (celular)**
  - Type: **BOOLEAN**
  - Definition: ¿Este hogar cuenta con teléfono móvil o celular? (Does this household have a mobile or cell phone?)

---

#### Accesso a los servicios de salud (Access to Health Services)

- **Horas en llegar al hospital (tsalud1_h)**
  - Type: **NUMERICO CONTINUO (CONTINUOUS NUMERIC)**
  - Definition: En caso de emergencia y considerando el medio de transporte que utilizaría, ¿cuánto tiempo cree que tardaría en llegar a un hospital en donde lo(a) pudieran atender? (In case of emergency and considering the means of transport you would use, how long do you think it would take to get to a hospital where you could be treated?)
- **Minutos en llegar al hospital (tsalud1_m)**
  - Type: **NUMERICO CONTINUO (CONTINUOUS NUMERIC)**
  - Definition: En caso de emergencia y considerando el medio de transporte que utilizaría, ¿cuánto tiempo cree que tardaría en llegar a un hospital en donde lo(a) pudieran atender? (Same definition as above, likely capturing minutes for precision).

---

## Vulnerabilidad climatica (Climatic Vulnerability)

- **Afectación por cambio climático (camb_clim)**
  - Type: **BOOLEAN**
  - Definition: De 2023 a la fecha, ¿en este hogar se vieron afectados por algún fenómeno climático como sequía, inundación, helada, incendio, huracán o tormenta tropical? (From 2023 to date, was this household affected by any climatic phenomenon such as drought, flood, frost, fire, hurricane, or tropical storm?)
- **Afectación por sequía (f_sequia)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha? (What was the last climatic phenomenon that affected them from 2023 to date?)
- **Afectación por inundación (f_inunda)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por helada (f_helada)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por incendio (f_incendio)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por huracán o ciclón (f_huracan)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por deslizamiento de tierras (f_desliza)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por otro fenómeno climático (f_otro)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Daños a la vivienda por fenómeno climático (af_viv)**
  - Type: **BOOLEAN**
  - Definition: Respecto al fenómeno climático referido, ¿cómo les afectó? (Regarding the referred climatic phenomenon, how did it affect them?)
- **Perdida de empleo por fenómeno climático (af_empleo)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Daño en su negocio por fenómeno climático (af_negocio)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Afectación a cultivos por fenómeno climático (af_cultivo)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Perdida de animales de trabajo por fenómeno climático (af_trabajo)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Afectaciones graves a su salud por fenómeno climático (af_salud)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Otro tipo de afectación por fenómeno climático (af_otro)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)

---

In [78]:
# Storing in a dictionary with dynamic key
dfs = {}
dfs[f"df_{file_name.split('.')[0]}"] = selected

In [79]:
# Printing the New_Dataframe new_df
print(f'Selected columns to df_{file_name.split('.')[0]}.csv from {file_name}')
dfs[f"df_{file_name.split('.')[0]}"]  # df_concentradohogar, df_erogaciones, df_gastoshogar, df_hogares, df_ingresos, df_viviendas

Selected columns to df_hogares.csv from hogares.csv


folioviv,acc_alim2,acc_alim7,acc_alim9,celular,tsalud1_h,tsalud1_m,camb_clim,f_sequia,f_inunda,f_helada,f_incendio,f_huracan,f_desliza,f_otro,af_viv,af_empleo,af_cultivo,af_trabajo,af_salud,af_otro
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
100001901,2,null,null,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100001902,2,null,null,1,1,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100001904,2,1,2,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100001905,2,2,2,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100002501,2,null,null,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3260593814,2,null,null,1,1,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null
3260593815,2,null,null,1,1,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null
3260593816,2,null,null,1,1,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null


In [80]:
# Build dynamic name for output file
output_file = f"selected_{file_name}"

# Save the DataFrame to CSV
dfs[f"df_{file_name.split('.')[0]}"].write_csv(output_file)

print(f"File saved: {output_file}")


File saved: selected_hogares.csv


#### TRATANDO AS VARIABLES BOOLEANAS


In [128]:
# Load selected_hogares.csv en PANDAS
df = pd.read_csv('selected_hogares.csv')

In [129]:
# Treating column 'camb_clim' to 0/1
df['camb_clim'] = df['camb_clim'].apply(lambda x: 1 if x == 1 else 0)

In [130]:
# Finding all columns with a single distinct value
single_value_columns = [col for col in df.columns if df[col].nunique() == 1]
df[single_value_columns].describe()

,f_sequia,f_inunda,f_helada,f_incendio,f_huracan,f_desliza,f_otro
count,8759.0,3391.0,423.0,220.0,2873.0,135.0,578.0
mean,1.0,2.0,3.0,4.0,5.0,6.0,7.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,2.0,3.0,4.0,5.0,6.0,7.0
25%,1.0,2.0,3.0,4.0,5.0,6.0,7.0
50%,1.0,2.0,3.0,4.0,5.0,6.0,7.0
75%,1.0,2.0,3.0,4.0,5.0,6.0,7.0
max,1.0,2.0,3.0,4.0,5.0,6.0,7.0


In [131]:
#import pandas as pd

# Assume single_value_columns is a list of column names you want to transform
# Example: single_value_columns = ['column_A', 'column_B', 'column_C']

# Define the set of values that should be transformed to 1
values_to_keep = [1, 2, 3, 4, 5, 6, 7]

for col in single_value_columns:
    # 1. df[col].isin(values_to_keep) checks if the value is 1, 2, 3, 4, 5, 6, or 7.
    #    It returns a Boolean Series (True/False).
    # 2. .astype(int) converts True to 1 and False to 0.
    #    Crucially, it handles NaN values by converting them to 0.

    df[col] = df[col].isin(values_to_keep).astype(int)

# --- IMPORTANT NOTE ON NaN HANDLING ---
# If your column originally had NaN values, this code transforms them to 0.
# If you want to PRESERVE NaN values while transforming 1-7 to 1, use this alternative:

# for col in single_value_columns:
#     # Use np.where to check for the condition and only assign 1 or 0 if it's not NaN
#     condition = df[col].isin(values_to_keep)
#     df[col] = np.where(condition, 1, np.where(df[col].isnull(), np.nan, 0))


df[single_value_columns].value_counts()


f_sequia  f_inunda  f_helada  f_incendio  f_huracan  f_desliza  f_otro
0         0         0         0           0          0          0         75889
1         0         0         0           0          0          0          8121
0         1         0         0           0          0          0          3025
          0         0         0           1          0          0          2584
                                          0          0          1           513
                    1         0           0          0          0           244
1         1         0         0           0          0          0           204
          0         0         0           1          0          0           151
                    1         0           0          0          0           145
0         0         0         1           0          0          0           144
          1         0         0           1          0          0            93
          0         0         0           0          1          0            80
1         0         0         1           0          0          0            55
                              0           0          0          1            36
                                                     1          0            18
0         0         0         0           1          1          0            17
          1         0         0           0          0          1            15
                    1         0           0          0          0            11
1         1         0         0           1          0          0             8
0         1         0         1           0          0          0             8
1         1         1         0           0          0          0             7
0         1         0         0           1          1          0             6
                                          0          1          0             6
          0         1         0           0          0          1             4
                    0         0           1          0          1             3
                                          0          1          1             3
                    1         0           1          0          0             3
                    0         1           1          0          0             3
1         1         0         1           0          0          0             2
          0         1         0           1          0          0             2
0         0         1         1           0          0          0             2
1         0         1         0           0          1          0             2
                    0         1           1          0          0             2
          1         0         0           0          1          0             2
0         1         0         0           1          0          1             1
          0         0         1           0          0          1             1
1         0         0         1           0          0          1             1
          1         1         0           0          1          0             1
                              1           0          0          0             1
                                                                1             1
Name: count, dtype: int64

In [132]:
# Finding all columns whose values are 1 or 2
flag_columns = [
    col for col in df.columns
    if df[col].min() == 1 and df[col].max() == 2
]

#flag_columns = [col for col in df.columns if df[col].isin([1, 2]).any()]
df[flag_columns].describe()

,acc_alim2,acc_alim7,acc_alim9,celular,af_viv,af_empleo,af_cultivo,af_trabajo,af_salud,af_otro
count,91414.000000,31544.000000,31544.000000,91414.000000,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000
mean,1.911086,1.700260,1.932222,1.060002,1.592786,1.908857,1.676779,1.925153,1.925346,1.772238
std,0.284621,0.458151,0.251369,0.237491,0.491331,0.287822,0.467721,0.263153,0.262840,0.419402
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000
50%,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [133]:
# TREATING flag_columns
# Create a mapping dictionary: {old_value: new_value}
mapping = {2: 0}

# Apply the replacement across all specified columns
# This operation automatically ignores and preserves NaN values.
df[flag_columns] = df[flag_columns].replace(mapping)

#flag_columns = [col for col in df.columns if df[col].isin([1, 2]).any()]
df[flag_columns].describe()

,acc_alim2,acc_alim7,acc_alim9,celular,af_viv,af_empleo,af_cultivo,af_trabajo,af_salud,af_otro
count,91414.000000,31544.000000,31544.000000,91414.000000,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000
mean,0.088914,0.299740,0.067778,0.939998,0.407214,0.091143,0.323221,0.074847,0.074654,0.227762
std,0.284621,0.458151,0.251369,0.237491,0.491331,0.287822,0.467721,0.263153,0.262840,0.419402
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [88]:
df.describe()

,folioviv,acc_alim2,acc_alim7,acc_alim9,celular,tsalud1_h,tsalud1_m,camb_clim,f_sequia,f_inunda,...,f_incendio,f_huracan,f_desliza,f_otro,af_viv,af_empleo,af_cultivo,af_trabajo,af_salud,af_otro
count,9.141400e+04,91414.000000,31544.000000,31544.000000,91414.000000,91414.000000,91414.000000,91414.000000,91414.000000,91414.0,...,91414.0,91414.0,91414.0,91414.0,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000
mean,1.615952e+09,0.088914,0.299740,0.067778,0.939998,0.427703,18.447415,0.169832,0.095817,0.0,...,0.0,0.0,0.0,0.0,0.407214,0.091143,0.323221,0.074847,0.074654,0.227762
std,9.321405e+08,0.284621,0.458151,0.251369,0.237491,1.282973,13.817174,0.375487,0.294341,0.0,...,0.0,0.0,0.0,0.0,0.491331,0.287822,0.467721,0.263153,0.262840,0.419402
min,1.000019e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.028396e+08,0.000000,0.000000,0.000000,1.000000,0.000000,5.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.565433e+09,0.000000,0.000000,0.000000,1.000000,0.000000,20.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.408016e+09,0.000000,1.000000,0.000000,1.000000,1.000000,30.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,3.260594e+09,1.000000,1.000000,1.000000,1.000000,90.000000,59.000000,1.000000,1.000000,0.0,...,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [134]:
# Saving selected_hogares_cleaned.csv
df.to_csv('selected_hogares_cleaned.csv', index=False)
